In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import polars as pl
import matplotlib.pyplot as plt
import seaborn as sns
# import dask.dataframe as dd

### Data Selection Step

In [2]:
train_labels = pl.read_csv('train_labels.csv')

In [3]:
# train_labels.head()

In [4]:
train_labels = train_labels.sample((len(train_labels)*0.2), seed = 16  )

In [5]:
train_labels.shape

(91782, 2)

In [6]:
print(train_labels.head())

shape: (5, 2)
┌─────────────────────────────────┬────────┐
│ customer_ID                     ┆ target │
│ ---                             ┆ ---    │
│ str                             ┆ i64    │
╞═════════════════════════════════╪════════╡
│ 46d09d19c7254974001a4fddb7b491… ┆ 0      │
│ 97cc37fe45453146c112982b37e1d1… ┆ 0      │
│ f2b8b6bdbe6055a3ab8ebfa64fcb58… ┆ 1      │
│ fdaa05e240a44f673007cdc111e09e… ┆ 1      │
│ 0fba5386c6d0157efa626f66ec3fa3… ┆ 1      │
└─────────────────────────────────┴────────┘


In [7]:
train_data = pl.read_csv('train_data.csv')

In [8]:
train_data.shape

(5531451, 190)

In [9]:
development_sample = train_labels.join(train_data, on='customer_ID', how='inner')

## Exploration

In [10]:
development_sample.head()

customer_ID,target,S_2,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,D_42,D_43,D_44,B_4,D_45,B_5,R_2,D_46,D_47,D_48,D_49,B_6,B_7,B_8,D_50,D_51,B_9,R_3,D_52,P_3,B_10,D_53,S_5,B_11,S_6,D_54,…,S_27,D_113,D_114,D_115,D_116,D_117,D_118,D_119,D_120,D_121,D_122,D_123,D_124,D_125,D_126,D_127,D_128,D_129,B_41,B_42,D_130,D_131,D_132,D_133,R_28,D_134,D_135,D_136,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145
str,i64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,f64,f64,str,f64,f64,str,str,str,str,str,f64,f64,f64,str,f64,f64,f64
"""0000099d6bd597052cdcda90ffabf5…",0,"""2017-03-09""",0.938469,0.001733,0.008724,1.006838,0.009228,0.124035,0.008771,0.004709,null,null,0.00063,0.080986,0.708906,0.1706,0.006204,0.358587,0.525351,0.255736,null,0.063902,0.059416,0.006466,0.148698,1.335856,0.008207,0.001423,0.207334,0.736463,0.096219,null,0.023381,0.002768,0.008322,1.001519,…,0.676922,0.007871,1.0,0.23825,0.0,4.0,0.23212,0.236266,0.0,0.70228,0.434345,0.003057,0.686516,0.00874,1.0,1.003319,1.007819,1.00008,0.006805,null,0.002052,0.005972,null,0.004345,0.001535,null,null,null,null,null,0.002427,0.003706,0.003818,null,0.000569,0.00061,0.002674
"""0000099d6bd597052cdcda90ffabf5…",0,"""2017-04-07""",0.936665,0.005775,0.004923,1.000653,0.006151,0.12675,0.000798,0.002714,null,null,0.002526,0.069419,0.712795,0.113239,0.006206,0.35363,0.521311,0.223329,null,0.065261,0.057744,0.001614,0.149723,1.339794,0.008373,0.001984,0.202778,0.720886,0.099804,null,0.030599,0.002749,0.002482,1.009033,…,0.822281,0.003444,1.0,0.247217,0.0,4.0,0.243532,0.241885,0.0,0.707017,0.430501,0.001306,0.686414,0.000755,1.0,1.008394,1.004333,1.008344,0.004407,null,0.001034,0.004838,null,0.007495,0.004931,null,null,null,null,null,0.003954,0.003167,0.005032,null,0.009576,0.005492,0.009217
"""0000099d6bd597052cdcda90ffabf5…",0,"""2017-05-28""",0.95418,0.091505,0.021655,1.009672,0.006815,0.123977,0.007598,0.009423,null,null,0.007605,0.068839,0.720884,0.060492,0.003259,0.33465,0.524568,0.189424,null,0.066982,0.056647,0.005126,0.151955,1.337179,0.009355,0.007426,0.206629,0.738044,0.134073,null,0.048367,0.010077,0.00053,1.009184,…,0.853498,0.003269,1.0,0.239867,0.0,4.0,0.240768,0.23971,0.0,0.704843,0.434409,0.003954,0.690101,0.009617,1.0,1.009307,1.007831,1.006878,0.003221,null,0.005681,0.005497,null,0.009227,0.009123,null,null,null,null,null,0.003269,0.007329,0.000427,null,0.003429,0.006986,0.002603
"""0000099d6bd597052cdcda90ffabf5…",0,"""2017-06-13""",0.960384,0.002455,0.013683,1.0027,0.001373,0.117169,0.000685,0.005531,null,null,0.006406,0.05563,0.723997,0.166782,0.009918,0.323271,0.530929,0.135586,null,0.08372,0.049253,0.001418,0.151219,1.339909,0.006782,0.003515,0.208214,0.741813,0.134437,null,0.030063,0.009667,0.000783,1.007456,…,0.844667,0.000053,1.0,0.24091,0.0,4.0,0.2394,0.240727,0.0,0.711546,0.436903,0.005135,0.687779,0.004649,1.0,1.001671,1.00346,1.007573,0.007703,null,0.007108,0.008261,null,0.007206,0.002409,null,null,null,null,null,0.006117,0.004516,0.0032,null,0.008419,0.006527,0.0096
"""0000099d6bd597052cdcda90ffabf5…",0,"""2017-07-16""",0.947248,0.002483,0.015193,1.000727,0.007605,0.117325,0.004653,0.009312,null,null,0.007731,0.038862,0.720619,0.14363,0.006667,0.231009,0.529305,null,null,0.0759,0.048918,0.001199,0.154026,1.341735,0.000519,0.001362,0.205468,0.691986,0.121518,null,0.054221,0.009484,0.006698,1.003738,…,0.811199,0.008724,1.0,0.247939,0.0,4.0,0.244199,0.242325,0.0,0.705343,0.437433,0.002849,0.688774,0.000097,1.0,1.009886,1.005053,1.008132,0.009823,null,0.00968,0.004848,null,0.006312,0.004462,null,null,null,null,null,0.003671,0.004946,0.008889,null,0.00167,0.008126,0.009827


In [11]:
development_sample.describe()

statistic,customer_ID,target,S_2,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,D_42,D_43,D_44,B_4,D_45,B_5,R_2,D_46,D_47,D_48,D_49,B_6,B_7,B_8,D_50,D_51,B_9,R_3,D_52,P_3,B_10,D_53,S_5,B_11,S_6,…,S_27,D_113,D_114,D_115,D_116,D_117,D_118,D_119,D_120,D_121,D_122,D_123,D_124,D_125,D_126,D_127,D_128,D_129,B_41,B_42,D_130,D_131,D_132,D_133,R_28,D_134,D_135,D_136,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145
str,str,f64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,f64,f64,str,f64,f64,str,str,str,str,str,f64,f64,f64,str,f64,f64,f64
"""count""","""1106348""",1.106348e6,"""1106348""",1.097211e6,1.106348e6,1.106348e6,1.105937e6,1.106348e6,902133.0,1.105937e6,1.105937e6,159608.0,774042.0,1.05098e6,1.106348e6,1.105936e6,1.106348e6,1.106348e6,862406.0,1.106348e6,962445.0,"""109711""",1.106295e6,1.106348e6,1.102056e6,476410.0,1.106348e6,1.106348e6,1.106348e6,1.10042e6,1.045731e6,1.106348e6,288893.0,1.106348e6,1.106348e6,1.106348e6,…,825232.0,1.070742e6,1.070742e6,1.070742e6,1.070742e6,1.070742e6,1.070742e6,1.070742e6,1.070742e6,1.070742e6,1.070742e6,1.070742e6,1.070742e6,1.070742e6,1.082792e6,1.106348e6,1.085768e6,1.085768e6,1.106212e6,"""14847""",1.085768e6,1.085768e6,"""109124""",1.097738e6,1.106348e6,"""38612""","""38612""","""38612""","""38612""","""38612""",1.085768e6,1.098123e6,1.085768e6,"""189232""",1.085768e6,1.098103e6,1.085768e6
"""null_count""","""0""",0.0,"""0""",9137.0,0.0,0.0,411.0,0.0,204215.0,411.0,411.0,946740.0,332306.0,55368.0,0.0,412.0,0.0,0.0,243942.0,0.0,143903.0,"""996637""",53.0,0.0,4292.0,629938.0,0.0,0.0,0.0,5928.0,60617.0,0.0,817455.0,0.0,0.0,0.0,…,281116.0,35606.0,35606.0,35606.0,35606.0,35606.0,35606.0,35606.0,35606.0,35606.0,35606.0,35606.0,35606.0,35606.0,23556.0,0.0,20580.0,20580.0,136.0,"""1091501""",20580.0,20580.0,"""997224""",8610.0,0.0,"""1067736""","""1067736""","""1067736""","""1067736""","""1067736""",20580.0,8225.0,20580.0,"""917116""",20580.0,8245.0,20580.0
"""mean""",null,0.249845,null,0.655761,0.153133,0.124832,0.621595,0.079656,0.226266,0.059783,0.13292,0.184003,0.154894,0.118729,0.17199,0.251728,0.082747,0.048316,0.475447,0.40718,0.382622,null,0.147372,0.187096,0.451636,0.173959,0.145326,0.189942,0.12136,0.180339,0.600668,0.226393,0.078323,0.089442,0.111732,0.244895,…,0.277287,0.157192,0.617864,0.277598,0.001207,2.314818,0.283387,0.279276,0.116457,0.532238,0.403645,0.051677,0.305891,0.084746,0.740137,0.10765,0.583685,0.438628,0.031733,null,0.202885,0.10398,null,0.046779,0.005796,null,null,null,null,null,0.179292,0.027051,0.164874,null,0.179169,0.052749,0.061982
"""std""",null,0.432924,null,0.245271,0.270201,0.21411,0.401399,0.228202,0.193244,0.204041,0.236015,0.227217,0.214451,0.221617,0.221473,0.242642,0.364635,0.203582,0.170149,0.235093,0.326293,null,0.959434,0.231845,0.497055,0.48357,0.241619,0.290361,0.212607,0.17447,0.171072,3.767505,0.208078,0.455773,0.210429,0.42703,…,0.326625,0.229995,0.48591,0.255306,0.034716,2.2929,0.255559,0.25611,0.320772,0.234069,0.236677,0.35503,0.227056,0.294076,0.536006,0.303516,0.49322,0.495579,0.21207,null,0.398426,0.296188,null,0.173968,0.028275,null,null,null,null,null,0.379375,0.146869,0.348628,null,0.379269,0.183638,0.19221
"""min""","""0000099d6bd597052cdcda90ffabf5…",0.0,"""2017-03-01""",-0.458955,5.0262e-9,-1.248558,6.2712e-7,2.6005e-9,-0.508883,1.3059e-9,1.4636e-8,-0.000327,5.4293e-7,2.2222e-9,1.6175e-8,4.7930e-7,1.2893e-8,8.2446e-9,-5.427248,-0.02662,-0.009612,"""0.00010502340608928296""",-0.004698,-0.746896,1.6356e-8,-3.672546,1.7920e-8,6.7261e-9,2.1348e-8,-0.007025,-1.51969,-0.002956,5.6781e-8,3.6067e-8,7.4633e-8,1.4336e-8,…,4.4171e-10,1.4635e-11,0.0,2.2281e-7,0.0,-1.0,1.4720e-7,2.3530e-7,0.0,-0.032074,0.000002,9.5234e-9,-0.04545,9.3151e-9,-1.0,5.4012e-8,3.7363e-9,2.4229e-10,2.0307e-9,"""0.000103147723058653""",2.1311e-9,2.6430e-8,"""-0.00010811408139725789""",1.2432e-9,1.1850e-8,"""-

In [12]:
development_sample = development_sample.with_columns(
    pl.col('S_2').str.strptime(pl.Date, '%Y-%m-%d')
)


In [13]:
development_sample.shape

(1106348, 191)

In [14]:
# Count the number of occurrences for each customer ID
customer_month_counts = development_sample.group_by('customer_ID').agg(
    months_of_data=pl.count()
)
# Merge this count back with the original dataframe to get the target variable for each customer
development_sample = development_sample.join(customer_month_counts, on='customer_ID')

/var/folders/_8/r6tmrd5913l149_tjy55m9xm0000gn/T/ipykernel_3826/4160444127.py:3: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
  months_of_data=pl.count()


In [15]:
print(development_sample.head())

shape: (5, 192)
┌──────────────┬────────┬────────────┬──────────┬───┬──────────┬──────────┬──────────┬─────────────┐
│ customer_ID  ┆ target ┆ S_2        ┆ P_2      ┆ … ┆ D_143    ┆ D_144    ┆ D_145    ┆ months_of_d │
│ ---          ┆ ---    ┆ ---        ┆ ---      ┆   ┆ ---      ┆ ---      ┆ ---      ┆ ata         │
│ str          ┆ i64    ┆ date       ┆ f64      ┆   ┆ f64      ┆ f64      ┆ f64      ┆ ---         │
│              ┆        ┆            ┆          ┆   ┆          ┆          ┆          ┆ u32         │
╞══════════════╪════════╪════════════╪══════════╪═══╪══════════╪══════════╪══════════╪═════════════╡
│ 0000099d6bd5 ┆ 0      ┆ 2017-03-09 ┆ 0.938469 ┆ … ┆ 0.000569 ┆ 0.00061  ┆ 0.002674 ┆ 13          │
│ 97052cdcda90 ┆        ┆            ┆          ┆   ┆          ┆          ┆          ┆             │
│ ffabf5…      ┆        ┆            ┆          ┆   ┆          ┆          ┆          ┆             │
│ 0000099d6bd5 ┆ 0      ┆ 2017-04-07 ┆ 0.936665 ┆ … ┆ 0.009576 ┆ 0.005492 ┆

In [16]:
# Group by the number of months of data and calculate the number of unique customer IDs and default rate
summary_table = development_sample.group_by('months_of_data').agg(
    Number_of_Observations=pl.col('customer_ID').n_unique(),
    Default_Rate=pl.col('target').mean()
).sort('months_of_data')

# Convert the summary table to a pandas DataFrame for display
summary_table_df = summary_table.to_pandas()

summary_table_df = summary_table_df.sort_values(by='months_of_data', ascending=False)

# Display the summary table
print(summary_table_df)


    months_of_data  Number_of_Observations  Default_Rate
12              13                   77113      0.232153
11              12                    2161      0.394262
10              11                    1274      0.464678
9               10                    1362      0.473568
8                9                    1283      0.442712
7                8                    1220      0.434426
6                7                    1011      0.420376
5                6                    1113      0.364780
4                5                     904      0.378319
3                4                     952      0.432773
2                3                    1160      0.356034
1                2                    1214      0.328666
0                1                    1015      0.340887


In [17]:
# Count the number of features in each category
categories = {
    'D_': 'Delinquency variables',
    'S_': 'Spend variables',
    'P_': 'Payment variables',
    'B_': 'Balance variables',
    'R_': 'Risk variables'
}

feature_counts = []
for prefix, category in categories.items():
    count = sum(feature.startswith(prefix) for feature in development_sample.columns)
    feature_counts.append({'Category': category, '# of features': count})

# Convert the list to a Pandas DataFrame
feature_counts_df = pd.DataFrame(feature_counts)

# Display the table
print(feature_counts_df)


                Category  # of features
0  Delinquency variables             96
1        Spend variables             22
2      Payment variables              3
3      Balance variables             40
4         Risk variables             28


In [18]:
categorical_columns = ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']

all_columns = [c for c in list(development_sample.columns) if c not in ['customer_ID','S_2']]

continuous_columns = [col for col in all_columns if col not in categorical_columns + ['target']]

In [19]:
for col in continuous_columns:
    development_sample = development_sample.cast({col: pl.Float64})

In [20]:
# Define the reference date (most recent date)
reference_date = pl.datetime(2018, 4, 30)

# # # Define a helper function to create aggregated features


# Define a function to create aggregated features
def create_aggregated_features(df, feature, months, reference_date):
    # Filter the DataFrame to include only the last 'months' months
    filtered_df = df.filter(pl.col('S_2') >= (reference_date - pl.duration(days=30*months)))
    
    # Group by 'customer_ID' and calculate aggregated features
    agg_df = filtered_df.group_by('customer_ID').agg([
        pl.col(feature).mean().alias(f"{feature}_Ave_{months}"),
        pl.col(feature).min().alias(f"{feature}_Min_{months}"),
        pl.col(feature).max().alias(f"{feature}_Max_{months}")
    ])
    
    return agg_df

# Define the list of numerical features for which you want to create aggregated features
numerical_features = continuous_columns  # Assuming 'continuous_columns' contains the list of numerical features

# Initialize an empty list to store the feature DataFrames
feature_dfs = []

In [21]:
# Initialize an empty list to store the feature DataFrames
feature_dfs = []

# Iterate over the numerical features and create aggregated features for different time periods
for feature in continuous_columns:
    for months in [6,12]:  # Specify the time periods you want to aggregate over
        feature_df = create_aggregated_features(development_sample, feature, months, reference_date)
        # Keep 'customer_ID' only in the first DataFrame
        if feature_dfs:
            feature_df = feature_df.drop('customer_ID')
        feature_dfs.append(feature_df)

# Combine all aggregated feature DataFrames into a single DataFrame
final_features_df = pl.concat(feature_dfs, how='horizontal')

# Add 'customer_ID' back to the final DataFrame by joining with the first DataFrame in feature_dfs
final_features_df = feature_dfs[0].select('customer_ID').join(final_features_df, on='customer_ID', how='inner')


In [22]:
df_cat = development_sample[categorical_columns + ['customer_ID', 'S_2']]
# for col in categorical_columns:
#     df_cat = df_cat.cast({col: pl.Float64})

In [23]:
df_cat = df_cat.to_dummies(columns = categorical_columns)

In [24]:
# Define a helper function to create aggregated features for binary categorical features
def create_categorical_aggregated_features(df, feature, months, reference_date):
    # Filter the DataFrame to include only the last 'months' months
    filtered_df = df.filter(pl.col('S_2') >= (reference_date - pl.duration(days=30*months)))
    
    # Group by 'customer_ID' and aggregate the feature
    agg_df = filtered_df.group_by('customer_ID').agg([
        (pl.col(feature).sum() / pl.count(feature)).alias(f"{feature}_Response_Rate_{months}"),
        (pl.col(feature).sum().map_elements(lambda x: 1 if x > 0 else 0)).alias(f"{feature}_Ever_Response_{months}")
    ])
    
    return agg_df

# Initialize a list to store the feature DataFrames
cat_feature_dfs = []

# Iterate over the categorical columns and create aggregated features
for feature in df_cat.columns:
    if feature in ['customer_ID', 'S_2']:
        continue
    for months in [6, 12]:
        cat_feature_dfs.append(create_categorical_aggregated_features(df_cat, feature, months, reference_date))

# Join all features into a single DataFrame
# Remove 'customer_ID' from all but the first DataFrame to avoid the DuplicateError
for i in range(1, len(cat_feature_dfs)):
    cat_feature_dfs[i] = cat_feature_dfs[i].drop('customer_ID')

final_cat_df = pl.concat(cat_feature_dfs, how='horizontal')

# print(final_cat_df)

sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredict

In [25]:
dataset = pl.concat([final_features_df, final_cat_df.drop("customer_ID"), development_sample.select(['customer_ID', 'S_2', 'target'] + continuous_columns).group_by('customer_ID').first().drop("customer_ID")], how = "horizontal")

In [26]:
dataset.shape

(91782, 1469)